## Functions
#### types of fucntions
    - operators and primitive functions
    - q-sql functions (i.e. select, update, upsert, insert, etc.)
    - user defined functions
    - adverbs or derived functions

In [1]:
// Infix and prefix notations
show 5 + 7 // infix notation
show +[5;7] // prefix notation
show (+) . 5 7 // apply notation

12
12
12


A q function is deliminated with {} and has optional [] for the declared variables
x,y,z are implicitly defined as inputs

In [2]:
{x*x} 5f

25f


In [3]:
{[int] int*int} 5f

25f


- prefix notation [] maybe needed when using longer expressions due to right to left evaluation
- Also good for code clerity
- Functions can ne nested

In [4]:
a:{[x;w]v:{[x;w] 3*w+x};v[x;2*w]-4f}
a[4f;3f]

26f


Functions and variables defined within the fucntion are local to the functions.

To declare global inside the function use ::

In [0]:
// v has no visibility from above
v

[0;31mv[0m: [0;31mv[0m

In [2]:
f:{[b]a::6;a+b}
show f[7];
a // global

13


6


Multivariate functions can be projected by specifying only part of the input params.

Projecton is related to currying in functional programming.

Monadic function can be a projection of dyadic funciton by fixing all except one input param

In [6]:
f:{x+y}
g:f[4;]
g[5]

9


### 3.1Namespace
A namespace is a context with which we resolve problems of unique naming. You can define variables and functions inside it.
It can be used to resolve local vs global variable avilability.

#### Limits in kdb/q:
- Input params for function (8 max)
- Constants (max 96)
- Global variables (32 max)
- Locals variables (24 max)
- More than 999 nested columns in splayed table

In [7]:
.namespace1.x:4f;
.namespace1.f:{5+x};
.namespace2.x:5;
.namespace2.f:{1f+log x};
// we can call from both
.namespace2.f[.namespace1.x]

2.386294


#### 3.1.01 .quantQ Namespace
In this book all the fucntions and variables defined are in this .quantQ namespace. All relavant funcitons are furthur listed in sub-namespaces (namespace with in the namespace) in logocal grouping.

In [11]:
\l quantQ-master\lib\quantQ_complex.q 
// Note: trying to load in the entire quantQ.q namespace results in error (Ram issue?)

In [9]:
// subnamespace complex for fourier transformation 
\f .quantQ.complex

`conjugate`dft`dft1k`dftTab`dftTs`div`exp`fractal`log`mandelbrot`mandelbrot_`..


### 3.2 The six adverbs
 - Kdb has nouns, verbs and adverbs
 - Nouns : variables and functions are nouns
 - syntactic structures and arectmatic and logical operators are verbs like {} [] or + -
 - Adverbs are higer order functions that modify the applied to lists. They return derived functions known as derivatives.

##### Adverbs
- 1) each
   -  1.2) each left \:
   -  1.3) each right /:
   -  1.4) cross product (/:\: (combination of each left and each right) or cross)
   -  1.5) each both '
   -  1.6) each prior ':
- 2) compose (')
- 3) Over/Fold /
- 4) Scan \
- 5) converge / \
- 6) cross product `/:\:` (or `cross` for cartisian product) 

In [10]:
// each modifies a monadic function, i.e. reverse
reverse each ((`a`b`c);(`d`e`f))

c b a
f e d


In [11]:
// without each
reverse((`a`b`c);(`d`e`f))

d e f
a b c


In [12]:
// Each left modifies a dyadic function to apply on each item in the left
("Hello ";"Greetings ";"Welcome "),\: "Mo"

"Hello Mo"
"Greetings Mo"
"Welcome Mo"


In [13]:
// Each right modifies a dyadic function to apply on each item in the right
"Hello ",/:("Alice";"Bob";"Mo")

"Hello Alice"
"Hello Bob"
"Hello Mo"


In [3]:
// usecase in creating an identity matrix
.quantQ.identity:{[n] `float$(til n)=/:til n}
.quantQ.identity[5]

1 0 0 0 0
0 1 0 0 0
0 0 1 0 0
0 0 0 1 0
0 0 0 0 1


In [15]:
// cross product or Cartesian product is the composition of an each right withy an each left
raze (4 5 6f) ,/:\:(5 6 7f)

4 5
4 6
4 7
5 5
5 6
5 7
6 5
6 6
6 7


In [16]:
(4 5 6f) cross (5 6 7f)

4 5
4 6
4 7
5 5
5 6
5 7
6 5
6 6
6 7


In [19]:
// cross is set, but /:\: is more versitile and can modify a monadic function
(4 5 6f) */:\:(5 6 7f)

20 24 28
25 30 35
30 36 42


In [20]:
// simmilar to dot notation
{x*y} . ((4 5 6f);(5 6 7f))

20 30 42f


In [19]:
// Each both, apllied pairwise to items on lists (of the same size)
show (4 5 6f) *'(5 6 7f)

20 30 42f


In [19]:
// returns 'length error when it's not the same `cardinality` (size)
(1 2 3) +'(4 5 6 7)

[0;31mlength[0m: [0;31mlength[0m

In [21]:
// good usecase of each both is to add columm to table
tbl:([sym:20?`A`B`C`D] market:20?`US`UK`CH`FR;price:20?500.00);
show tbl1:select symCount:count i by sym from tbl where market=`US;
show tbl2:select avgPrice:avg price by sym from tbl where market=`US;

(0!tbl1) ,' select avgPrice from tbl2 // pairwise simple join

sym| symCount
---| --------
A  | 2       
B  | 2       
D  | 1       
sym| avgPrice
---| --------
A  | 217.5092
B  | 180.7113
D  | 289.2601


sym symCount avgPrice
---------------------
A   2        217.5092
B   2        180.7113
D   1        289.2601


In [21]:
//note in many instances q's build in function is sufficent
(sum 1 2 3f * ' 3 4 5f) ~ sum 1 2 3f * 3 4 5f

1b


In [22]:
// $ - built in efficent dot product
(1 2 3f)$(3 4 5f)

26f


Here we show a more complex example of each both. It'll use a the calc function which takes in a nested list and a scaler and applies it  to create a new column

In [24]:
n:100
t:([]a:-2.5 + (n#3) ?\: 5f; b:n?10)
t

a                                  b
------------------------------------
2.177047    -1.900767   2.430673   5
-1.006319   2.422636    0.5240541  5
-0.9545289  1.473937    0.3211452  3
-1.228041   -0.1719607  -1.713999  4
0.7161745   1.578856    -0.5339601 5
1.545387    0.5566162   -2.091224  7
-1.299017   0.3921042   0.2827921  7
-0.5552802  1.071177    1.069554   7
-0.06418705 2.143434    0.806258   6
1.957702    -1.646659   -0.3918882 5
1.517074    -0.7727823  0.4910278  1
0.9617338   -0.7733395  0.4341187  1
-0.3191604  0.637679    0.05999989 9
-1.308504   1.86228     1.521763   9
0.3704817   0.8489581   0.4435739  0
0.3392884   -0.5277085  2.128066   6
0.8015667   -0.1022303  2.377889   4
-1.785298   -0.02305322 0.7883459  1
0.7359993   -1.497223   -0.7002882 0
1.09645     -0.4231628  1.655005   3
..


In [30]:

calc:{$[all 0 < x;x xexp reciprocal y;x xexp y]}

calc[t[`a][0];t[`b][0]]

48.90336 -24.811 84.84628


In [32]:
update c: calc'[a;b] from t

a                                  b c                                        
------------------------------------------------------------------------------
2.177047    -1.900767   2.430673   5 48.90336       -24.811      84.84628     
-1.006319   2.422636    0.5240541  5 -1.031995      83.4527      0.03952578   
-0.9545289  1.473937    0.3211452  3 -0.8696956     3.202112     0.03312108   
-1.228041   -0.1719607  -1.713999  4 2.27432        0.0008744126 8.630628     
0.7161745   1.578856    -0.5339601 5 0.1884058      9.810992     -0.04340542  
1.545387    0.5566162   -2.091224  7 21.05041       0.01655355   -174.9059    
-1.299017   0.3921042   0.2827921  7 -6.241698      0.001424987  0.0001446342 
-0.5552802  1.071177    1.069554   7 -0.01627742    1.618187     1.601103     
-0.06418705 2.143434    0.806258   6 6.99334e-008   96.97508     0.2746908    
1.957702    -1.646659   -0.3918882 5 28.75632       -12.10648    -0.009242963 
1.517074    -0.7727823  0.4910278  1 1.517074       

What about if column b was also nested?

In [33]:
t:([]a:-2.5 + (n#3) ?\: 5f; b:-2.5 + (n#3) ?\: 5f)

In [34]:
t

a                               b                                 
------------------------------------------------------------------
1.866501  1.567985   1.781887   1.147589   0.9143171   -1.940201  
0.3259302 1.637819   -0.4991273 -1.48574   -1.312727   -1.958312  
-2.268402 -1.120815  -0.9944251 -1.20311   0.8492526   0.8379796  
0.418897  0.4538928  0.865698   -0.4398862 -1.182655   -2.397292  
-1.817437 1.688792   2.392992   -0.3492527 -2.240802   -0.01130007
-2.313862 -0.5138037 -1.035665  -0.1099137 2.047722    1.039351   
-2.314594 2.167489   -2.002316  -1.435641  0.6231003   0.7126012  
-1.674711 2.246816   1.689958   0.3332368  -1.016841   -0.1705641 
1.993304  1.03037    -1.466092  -0.8256014 -2.254543   2.028039   
0.2342439 -0.4484026 -2.42046   2.13775    0.44809     2.476484   
-2.370333 -0.4071474 2.006625   -2.37483   0.9450842   -2.261345  
-1.563257 0.2746066  1.547083   2.001417   -2.224359   -2.275522  
-1.936092 1.178785   2.21097    0.5984827  -0.7861934  1.38343

In [35]:
update c: calc'[a;b] from t

a                               b                                  c         ..
-----------------------------------------------------------------------------..
1.866501  1.567985   1.781887   1.147589   0.9143171   -1.940201   1.72255   ..
0.3259302 1.637819   -0.4991273 -1.48574   -1.312727   -1.958312   5.288958  ..
-2.268402 -1.120815  -0.9944251 -1.20311   0.8492526   0.8379796             ..
0.418897  0.4538928  0.865698   -0.4398862 -1.182655   -2.397292   7.228852  ..
-1.817437 1.688792   2.392992   -0.3492527 -2.240802   -0.01130007           ..
-2.313862 -0.5138037 -1.035665  -0.1099137 2.047722    1.039351              ..
-2.314594 2.167489   -2.002316  -1.435641  0.6231003   0.7126012             ..
-1.674711 2.246816   1.689958   0.3332368  -1.016841   -0.1705641            ..
1.993304  1.03037    -1.466092  -0.8256014 -2.254543   2.028039    0.5658113 ..
0.2342439 -0.4484026 -2.42046   2.13775    0.44809     2.476484    0.04492711..
-2.370333 -0.4071474 2.006625   -2.37483

#### Each Prior
a dyadic operatioon on each item of a list with it's predecessor

In [46]:
4f {show (x;y);x-y xexp 2}': 5 6 7f

5 4f
6 5f
7 6f


-11 -19 -29f


#### Compose(')
The composition of a multivariate function f[a,b,c,...] with a monadic function g[x], expressed as g(f) is expressed as '[g;f]

In [47]:
f:{[a;b;c;d] a+(2*b)+(3*c)+4*d}; // multivariate function
g:{log x}; // monadic function
d:('[g;f]); // applies the multivariate first then monadic
d[5;6;7;8]

4.248495


#### Over and Fold /
Over adverb / modified a dyadic function to accumulate the results over a list.
Let us denote $y_{i} , i \in \mathbb{N}^{*}_{+}$. The over adverb recursivly applies elements of y recursivly.

In [48]:
4f {x-y xexp 2}/(5 6 7f)

-106f


In [1]:
// equvalint to
(((4f-5f xexp 2)-6f xexp 2)-7f xexp 2)

-106f


In [4]:
// can also work without spliting the list
({x-y xexp 2}/) (4 5 6 7f)

-106f


The over adverb is called fold, when the function has more than 2 arguments

In [5]:
({z+x-y xexp 2}/)[1 3f;4 5f;2f]

-36 -34f


In [8]:
// equivalant to
(2f + (2f+1f-4f xexp 2)-5f xexp 2; 2f + (2f+3f-4f xexp 2)-5f xexp 2)

-36 -34f


The adverb fold can be thought of in a map-reduce context, where we fold (aka reduce) the result set.

#### Scan \

It's simmilar to the behavior of over, except it shows the intermediate results


In [10]:
({x-y xexp 2}\) (4 5 6 7f)

4 -21 -57 -106f


#### EMA: The Exponential Moving Average

Lets show how scan can be used to compuite this very common itterative function for which we want the intermedeate results.

Given a vector $x_{0}, ... x_{n}$ , and weight $\lambda$ , the EMA is:
   $EMA_{i} (\lambda) = x_{i-1} * (1-\lambda ) + EMA_{i-1} * \lambda$

In [1]:
\l quantQ-master\lib\quantQ_stats.q
.quantQ.stats.expma1

{[lambda;vector]
    / lambda -- memory
    / vector -- data
    {[x;y;z] (x*y)+z}\[ first vector; 1 - lambda; vector * lambda]
 }


In [2]:
.quantQ.stats.expma1[.2;10 20 30]

10 12 15.6


In [3]:
.quantQ.stats.expma

{[lambda;vector]
    // lambda -- memory
    // vector -- data
    :(first vector)(1-lambda)\vector*lambda;
 }


#### Converge
The over and scan adverbs can also be used to itterate over a list until convergence or until a loop is detected.

#### converge-repeate
applies the function f x times with initial value y. If x is a boolian function to indicate a condition on recursive iteration, the expression is equvalant to a while loop.

In [12]:
2 {x*x-1}/(2 4 3 2 5f)

2 132 30 2 380f


In [17]:
{x<20}{show x;x*x}/2

2
4
16


256


#### converge-iterate
The converge-itterate pattern is a simmilar to the converge-repete pattern describe in the previous section but using scan adverb to display all intermediary. Hence, the previous example applied using the scan \ adverb.

In [8]:
2 {x*x-1}\(2 4 3 2 5f)

2 4   3  2 5  
2 12  6  2 20 
2 132 30 2 380


In [19]:
{x<20}{show x;x*x}\2

2
4
16


2 4 16 256


### 3.3 Apply
##### @ (apply)
The verb @ is used to apply a multivariate mapping to a list L with the syntax: @[L;I;f]. The following expression applies the function cosine on the first and third elements of the list; the second element 1.570796 being left unchanged.

In [20]:
@[(0.7853982; 1.570796; 3.141592654);0 2;cos]

0.7071068 1.570796 -1


When applied to dyadic function. @[L;I;f;a] applies a dyadic function f on the I-th element of a list L with seconds arguments a.

In [21]:
@[1 2 3 4f; 0 2;{x+2*y};6 7f]

13 2 17 4f


##### . (apply)
A function of arity N applied to a list x of size N is executed with the . verb as .[f;x] or f . x.

In [23]:
{[a;b;c;d] a+2*b+3*c-d} . (4;5;3;7)

-10


In [24]:
// monadic
show {x*x} . enlist 5;
show {x*x}@5;

25
25


applied to lists, the verb . indexes a list at depth

In [29]:
L:((`a;4);(`c;5);(`d;7;(8;6;9f)))
L . 2 2 2

9f


In [33]:
show L;
// get a row in the list by
show L[1];
// get the column in a nested list by
L . (::;1)

(`a;4)
(`c;5)
(`d;7;(8;6;9f))
`c
5


4 5 7


In [34]:
// equvilant notaition
L[::;1]

4 5 7


The verb . can also be applied to dictionaries

In [1]:
d:`a`b`c!((4f;5f;6f);3;`t);
d . (`a;1)

5f


In [2]:
d . (`a;::)

4 5 6f


 This .[L;I;f] applies function f to Ith index of L and returns it, keeping the list unchanged.
While .[`L;I;f] modifies the existing list (by reference)

In [6]:
L:((`a;4);(`c;5);(`d;7;(8;6;9f)));
show .[L;2 2;log];
L

(`a;4)
(`c;5)
(`d;7;2.079442 1.791759 2.197225)


(`a;4)
(`c;5)
(`d;7;(8;6;9f))


In [7]:
show .[`L;2 2;log];
L

`L


(`a;4)
(`c;5)
(`d;7;2.079442 1.791759 2.197225)


### 3.4 Protected Evaluation
The concept of try catch, or protected evaluation, is inplimented with the samew @ and . verbs. The trap function has the syntax @[f;arg;err] where f is a monadic function, and arg its argument. For multivariate funcitons, use .[f;args;err] where args are a list of arguments.  

In [22]:
f:{@[sqrt;x;`err]}

In [23]:
f[9f]

3f


In [25]:
f[`a]

`err


In [27]:
mult:{.[*;(x;y);`err]};
show mult[4;5];
mult[4;"toto"]

20


`err


In [28]:
// customize error handeler
mult:{.[*;(x;y);{"Error is: ",x}]};
mult[4;"toto"]

"Error is: type"


### 3.5 Vector Operations
In this section we will present a few more native functions which cover basic time series operations. We split the functions into two classes: aggregators.